In [48]:
import pandas as pd
import pickle
from gexf import Gexf
import re
from lxml import etree

In [10]:
df = pd.read_csv("../DataSet/douban_boutique/movie.csv", encoding='gbk')
df.head()

,title,types,score,director,writer,actor,account,country,label,flim_version
0,这个杀手不太冷,"剧情, 动作, 犯罪",9.4,吕克·贝松 Luc Besson,吕克·贝松 Luc Besson,"让·雷诺, 娜塔莉·波特曼, 加里·奥德曼, 丹尼·爱罗, 彼得·阿佩尔, 迈克尔·巴达鲁科...",1088567,法国,1023.25298,2
1,七武士,"动作, 冒险, 剧情",9.2,黑泽明 Akira Kurosawa,"黑泽明 Akira Kurosawa, 小国英雄 Hideo Oguni, 桥本忍 Shin...","三船敏郎, 志村乔, 稻叶义男, 宫口精二, 千秋实, 加东大介, 木村功",91163,日本,83.86996,6
2,蝙蝠侠：黑暗骑士,"剧情, 动作, 科幻, 犯罪, 惊悚",9.1,克里斯托弗·诺兰 Christopher Nolan,"乔纳森·诺兰 Jonathan Nolan, 克里斯托弗·诺兰 Christopher No...","克里斯蒂安·贝尔, 希斯·莱杰, 艾伦·艾克哈特, 迈克尔·凯恩, 玛吉·吉伦哈尔, 加里·...",423487,"美国, 英国",385.37317,1
3,指环王3：王者无敌,"剧情, 动作, 奇幻, 冒险",9.1,彼得·杰克逊 Peter Jackson,"弗兰·威尔士 Fran Walsh, Philippa Boyens, 彼得·杰克逊 Pet...","维果·莫腾森, 伊莱贾·伍德, 西恩·奥斯汀, 丽芙·泰勒, 伊恩·麦克莱恩, 奥兰多·布鲁...",350209,"美国, 新西兰",318.69019,3
4,搏击俱乐部,"剧情, 动作, 悬疑, 惊悚",9.0,大卫·芬奇 David Fincher,"恰克·帕拉尼克 Chuck Palahniuk, Jim Uhls","爱德华·诺顿, 布拉德·皮特, 海伦娜·伯翰·卡特, 扎克·格雷尼尔, 米特·洛夫, 杰瑞德...",465674,"美国, 德国",419.10660,1


### 生成types集合

In [12]:
type_set = set()
for i in range(df.shape[0]):
    type_list = df.iloc[i,1].split(', ')
    for i in range(len(type_list)):
        type_list[i] = type_list[i].strip("\'")
    type_set = type_set | set(type_list)
print(type_set)

{'灾难', '战争', '爱情', 'Adult', '古装', '悬念', '西部', '情色', '悬疑', '动画', '历史', 'Reality-TV', '运动', '惊悚', '音乐', '剧情', '传记', '儿童', '奇幻', '科幻', '动作', '恐怖', '歌舞', '犯罪', '家庭', '黑色电影', '同性', '武侠', '喜剧', '惊栗', '鬼怪', '冒险'}


In [15]:
with open('type_set.pkl','wb') as f:
    pickle.dump(type_set,f)

### 统计演员出场次数

In [11]:
actor_dic = dict()
for i in range(df.shape[0]):
    actor_str = df.iloc[i,5]
    actor_list = actor_str.split(',')
    #print(actor_list)
    for a in actor_list:
        a = a.strip()
        if a in actor_dic:
            actor_dic[a] += 1
        else:
            actor_dic[a] = 1

actor_dic

{'让·雷诺': 33,
 '娜塔莉·波特曼': 35,
 '加里·奥德曼': 33,
 '丹尼·爱罗': 7,
 '彼得·阿佩尔': 2,
 '迈克尔·巴达鲁科': 4,
 '艾伦·格里尼': 2,
 '伊丽莎白·瑞根': 1,
 '卡尔·马图斯维奇': 1,
 '弗兰克·赛格': 1,
 '麦温': 4,
 '乔治·马丁': 3,
 '罗伯特·拉萨多': 2,
 '亚当·布斯奇': 2,
 '马里奥·托迪斯科': 1,
 '萨米·纳塞利': 7,
 '三船敏郎': 20,
 '志村乔': 14,
 '稻叶义男': 3,
 '宫口精二': 7,
 '千秋实': 6,
 '加东大介': 5,
 '木村功': 2,
 '克里斯蒂安·贝尔': 28,
 '希斯·莱杰': 13,
 '艾伦·艾克哈特': 21,
 '迈克尔·凯恩': 43,
 '玛吉·吉伦哈尔': 15,
 '摩根·弗里曼': 46,
 '莫尼克·加布里埃拉·库尔内': 4,
 '罗恩·迪恩': 1,
 '基里安·墨菲': 21,
 '黄经汉': 8,
 '内斯特·卡博内尔': 2,
 '埃里克·罗伯茨': 10,
 '里特奇·科斯特': 2,
 '安东尼·迈克尔·豪尔': 6,
 '基斯·斯扎拉巴基克': 2,
 '柯林·麦克法兰': 4,
 '约书亚·哈尔托': 3,
 '美琳达·麦格劳': 1,
 '内森·甘宝': 4,
 '迈克尔·维约': 1,
 '迈克尔·斯托扬诺夫': 1,
 '威廉·斯米利': 1,
 '丹尼·高德林': 1,
 '迈克尔·加·怀特': 5,
 '马修·奥尼尔': 1,
 '威廉·菲德内尔': 17,
 '欧鲁米基·欧拉乌米': 1,
 '格雷格·比姆': 1,
 '爱利克·赫尔曼': 1,
 '毕崔斯·罗森': 2,
 '文森佐·尼克力': 1,
 '陈冠希': 24,
 '维果·莫腾森': 19,
 '伊莱贾·伍德': 26,
 '西恩·奥斯汀': 13,
 '丽芙·泰勒': 17,
 '伊恩·麦克莱恩': 19,
 '奥兰多·布鲁姆': 20,
 '凯特·布兰切特': 38,
 '米兰达·奥图': 12,
 '安迪·瑟金斯': 18,
 '雨果·维文': 21,
 '多米尼克·莫纳汉': 7,
 '比利·博伊德': 4,
 '马尔顿·索克斯': 14,
 '卡尔·厄本

In [12]:
sort_list = sorted(actor_dic.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
sort_list[:77]

[('林雪', 128),
 ('曾志伟', 127),
 ('刘德华', 115),
 ('黄秋生', 106),
 ('吴君如', 99),
 ('任达华', 99),
 ('吴镇宇', 84),
 ('梁家辉', 82),
 ('古天乐', 80),
 ('成龙', 77),
 ('吴孟达', 71),
 ('杜汶泽', 67),
 ('周润发', 67),
 ('午马', 67),
 ('刘青云', 66),
 ('梁朝伟', 65),
 ('陈小春', 60),
 ('许绍雄', 60),
 ('舒淇', 60),
 ('罗伯特·德尼罗', 60),
 ('张达明', 60),
 ('洪金宝', 59),
 ('大卫·苏切', 59),
 ('塞缪尔·杰克逊', 59),
 ('吴彦祖', 59),
 ('成奎安', 57),
 ('郑伊健', 56),
 ('布鲁斯·威利斯', 56),
 ('罗家英', 55),
 ('张曼玉', 54),
 ('陈百祥', 53),
 ('周星驰', 53),
 ('林家栋', 52),
 ('张学友', 52),
 ('苑琼丹', 50),
 ('约翰尼·德普', 50),
 ('尼古拉斯·凯奇', 50),
 ('李灿森', 49),
 ('张国荣', 49),
 ('高捷', 48),
 ('吴耀汉', 48),
 ('郑中基', 47),
 ('罗兰', 47),
 ('秦沛', 47),
 ('朱丽安·摩尔', 47),
 ('张家辉', 47),
 ('摩根·弗里曼', 46),
 ('连姆·尼森', 45),
 ('谷德昭', 45),
 ('浅野忠信', 45),
 ('方中信', 45),
 ('马特·达蒙', 44),
 ('郑则仕', 44),
 ('狄龙', 44),
 ('冯淬帆', 44),
 ('元华', 44),
 ('余文乐', 44),
 ('钱嘉乐', 43),
 ('迈克尔·凯恩', 43),
 ('郭富城', 42),
 ('谢霆锋', 42),
 ('约翰·古德曼', 42),
 ('林正英', 42),
 ('徐锦江', 42),
 ('妮可·基德曼', 42),
 ('欧文·威尔逊', 41),
 ('李连杰', 41),
 ('惠英红', 41),
 ('妻夫木聪',

In [13]:
with open('../DataSet/douban_more/sort_list.pkl', 'wb') as f:
    pickle.dump(sort_list,f)

### *读入演员列表

In [9]:
with open('../DataSet/douban_more/sort_list.pkl', 'rb') as f:
    sort_list = pickle.load(f)

EOFError: Ran out of input

### 生成gexf文件(test)

In [2]:
gexf = Gexf("LauZyHou","Actor Relation")
graph=gexf.addGraph("undirected","static","Actor Relation")

In [3]:
atr1 = graph.addNodeAttribute("Modularity Class",type="integer")

In [4]:
tmp = graph.addNode("0","刘德华")
tmp.addAttribute(atr1,"50")

tmp = graph.addNode("1","张学友")
tmp.addAttribute(atr1,"40")

tmp = graph.addNode("2","古巨基")
tmp.addAttribute(atr1,"70")

In [5]:
graph.addEdge("0","0","1",weight='1')
graph.addEdge("1","0","2",weight='1')
graph.addEdge("2","1","0",weight='1')

In [7]:
with open('test.gexf','wb') as f:
    gexf.write(f)

### 生成演员关系的gexf文件

In [56]:
gexf = Gexf("LauZyHou","Actor Relation")
graph=gexf.addGraph("undirected","static","Actor Relation")

atr1 = graph.addNodeAttribute("Modularity Class","0",type="integer")

In [57]:
# 创建结点
for i,(name,val) in enumerate(sort_list[:77]):
    tmp = graph.addNode(str(i),name)
    tmp.addAttribute(atr1,str(i//11)) # 这里77个结点分成7类

In [58]:
_id = 0
# 创建边，对任意两个演员搜索是否合作过
for actA in range(76): # 演员A
    nameA = sort_list[actA][0]
    for actB in range(actA,77): # 演员B
        nameB = sort_list[actB][0]
        hz_num = 0 # 合作次数
        for i in range(df.shape[0]): # 对每个电影，匹配看看这两个演员有没有合作过
            actstr = df.iloc[i,5]
            if (re.search(nameA+r".*"+nameB,actstr) is not None) or (re.search(nameB+r".*"+nameA,actstr) is not None):
                hz_num += 1
        if hz_num>0: # 如果合作过，todo生成边
            graph.addEdge(str(_id),str(actA),str(actB),weight=str(hz_num))
            _id += 1

In [59]:
with open('actor_relation.gexf','wb') as f:
    gexf.write(f)

In [60]:
with open('actor_relation.pkl','wb') as f:
    pickle.dump(gexf,f)

### 写入viz:size标签

In [61]:
gexf_xml = gexf.getXML()
for gexf_elem in gexf_xml:
    if gexf_elem.tag == 'graph':
        for gexf_nodes_links in gexf_elem:
            if gexf_nodes_links.tag == 'nodes':
                print("dealing with nodes viz")
                for node in gexf_nodes_links:
                    tmp_id = int(node.get('id'))
                    size_value = str(sort_list[tmp_id][1])
                    size = etree.SubElement(node, '{%s}size' % gexf.viz)
                    size.set('value', size_value)

dealing with nodes viz


In [62]:
with open('actor_relation2.gexf','wb') as f:
    f.write(etree.tostring(gexf_xml, pretty_print=True, encoding='utf-8', xml_declaration=True))

In [63]:
with open('actor_relation2.pkl','wb') as f:
    pickle.dump(gexf,f)

### *测试周星驰吴孟达

In [55]:
k=0
for i in range(df.shape[0]): # 对每个电影，匹配看看这两个演员有没有合作过
    actstr = df.iloc[i,5]
    if (re.search("周星驰"+r".*"+"吴孟达",actstr) is not None) or (re.search("吴孟达"+r".*"+"周星驰",actstr) is not None):
        k += 1

k

25